In [ ]:
pip install tensorflow pandas numpy matplotlib scikit-learn


In [ ]:

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:

csv_file_path = 'C:/Users/Suvodeep Banerjee/Downloads/train.csv'
train_image_dir = 'C:/Users/Suvodeep Banerjee/Downloads/train_images/'
test_image_dir = 'C:/Users/Suvodeep Banerjee/Downloads/test_images/'
train_tfrecords_dir = 'C:/Users/Suvodeep Banerjee/Downloads/train_tfrecords/'


In [6]:
label_mapping = {
    0: "Cassava Bacterial Blight (CBB)",
    1: "Cassava Brown Streak Disease (CBSD)",
    2: "Cassava Green Mottle (CGM)",
    3: "Cassava Mosaic Disease (CMD)",
    4: "Healthy"
}

In [7]:

df = pd.read_csv(csv_file_path)
df['image_path'] = df['image_id'].apply(lambda x: os.path.join(train_image_dir, x))
images = df['image_path'].values
labels = df['label'].values


In [47]:

le = LabelEncoder()
labels_encoded = le.fit_transform(labels)


train_df = pd.DataFrame({
    'image_path': X_train,
    'label': y_train
})

val_df = pd.DataFrame({
    'image_path': X_val,
    'label': y_val
})


In [67]:

train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [49]:
val_datagen = ImageDataGenerator(rescale=1.0/255.0)

In [50]:

df['label'] = df['label'].astype(str)

In [51]:

images = df['image_path'].values
labels = df['label'].values


In [52]:
# Data Preprocessing and Augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1.0/255.0)

In [54]:
# Train generator
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df, 
    x_col='image_path',
    y_col='label',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

# Validation generator
val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,  
    x_col='image_path',
    y_col='label',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)


Found 17117 validated image filenames belonging to 5 classes.
Found 4280 validated image filenames belonging to 5 classes.


In [55]:
#CNN Model
model = keras.Sequential([
    layers.Input(shape=(128, 128, 3)),  
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(label_mapping), activation='softmax')  # 5 classes
])

In [56]:

# Compile
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [57]:

epochs = 10 
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs,
    steps_per_epoch=len(X_train) // 32,
    validation_steps=len(X_val) // 32
)

C:\Users\Suvodeep Banerjee\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
534/534 ━━━━━━━━━━━━━━━━━━━━ 299s 554ms/step - accuracy: 0.6189 - loss: 1.1507 - val_accuracy: 0.6320 - val_loss: 1.0292
Epoch 2/10
534/534 ━━━━━━━━━━━━━━━━━━━━ 0s 315us/step - accuracy: 0.6875 - loss: 1.0766 - val_accuracy: 0.6250 - val_loss: 0.9535
Epoch 3/10


C:\Users\Suvodeep Banerjee\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


534/534 ━━━━━━━━━━━━━━━━━━━━ 294s 550ms/step - accuracy: 0.6396 - loss: 1.0105 - val_accuracy: 0.6414 - val_loss: 1.0600
Epoch 4/10
534/534 ━━━━━━━━━━━━━━━━━━━━ 0s 280us/step - accuracy: 0.6562 - loss: 0.9941 - val_accuracy: 0.6250 - val_loss: 0.9168
Epoch 5/10
534/534 ━━━━━━━━━━━━━━━━━━━━ 291s 545ms/step - accuracy: 0.6421 - loss: 0.9834 - val_accuracy: 0.6574 - val_loss: 0.9486
Epoch 6/10
534/534 ━━━━━━━━━━━━━━━━━━━━ 0s 238us/step - accuracy: 0.8438 - loss: 0.6911 - val_accuracy: 0.6667 - val_loss: 0.9970
Epoch 7/10
534/534 ━━━━━━━━━━━━━━━━━━━━ 303s 567ms/step - accuracy: 0.6543 - loss: 0.9466 - val_accuracy: 0.6696 - val_loss: 0.9227
Epoch 8/10
534/534 ━━━━━━━━━━━━━━━━━━━━ 0s 371us/step - accuracy: 0.5938 - loss: 0.9130 - val_accuracy: 0.7500 - val_loss: 0.7754
Epoch 9/10
534/534 ━━━━━━━━━━━━━━━━━━━━ 530s 994ms/step - accuracy: 0.6636 - loss: 0.9115 - val_accuracy: 0.6711 - val_loss: 0.8956
Epoch 10/10
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 526us/step - accuracy: 0.8750 - loss: 0.4956 - va

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array

label_mapping = {
    0: "Cassava Bacterial Blight (CBB)",
    1: "Cassava Brown Streak Disease (CBSD)",
    2: "Cassava Green Mottle (CGM)",
    3: "Cassava Mosaic Disease (CMD)",
    4: "Healthy"
}

def predict_images(image_path_or_dir):
   
    if os.path.isdir(image_path_or_dir):
       
        image_files = [f for f in os.listdir(image_path_or_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
    else:
        image_files = [image_path_or_dir]
    
    for image_file in image_files:
        
        if os.path.isdir(image_path_or_dir):
            image_path = os.path.join(image_path_or_dir, image_file)
        else:
            image_path = image_file
        
       
        img = load_img(image_path, target_size=(128, 128)) 
        img_array = img_to_array(img) / 255.0  
        img_array = np.expand_dims(img_array, axis=0) 

       
        prediction = model.predict(img_array)
        predicted_class = np.argmax(prediction, axis=1)[0]  
        predicted_label = label_mapping[predicted_class]

        
        plt.imshow(img)
        plt.title(f'Predicted: {predicted_label}')
        plt.axis('off')
        plt.show()

# Example usage for single image
predict_images(r"C:\Users\Suvodeep Banerjee\Downloads\test_images\test11.jpg")



NameError: name 'model' is not defined